In [2]:
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
p_a=pd.read_csv('/home/shochcho/Documents/Thermal_hydraulics_and_neutronics_coupling-main/cross section data/Pu-239 absorption.csv')
p_f=pd.read_csv('/home/shochcho/Documents/Thermal_hydraulics_and_neutronics_coupling-main/cross section data/Pu-239 fission.csv')
p_s=pd.read_csv('/home/shochcho/Documents/Thermal_hydraulics_and_neutronics_coupling-main/cross section data/Pu-239 scattering.csv')

atom_density=6.02*10**23*19.84/239


In [4]:
x=10**-6*np.array(p_f['Incident energy'])
y=10**-24*atom_density*np.array(p_f['Cross section'])
pf=interp1d(x,y)

x=10**-6*np.array(p_a['Incident energy'])
y=10**-24*atom_density*np.array(p_a['Cross section'])
pa=interp1d(x,y)

x=10**-6*np.array(p_s['Incident energy'])
y=10**-24*atom_density*np.array(p_s['Cross section'])
ps=interp1d(x,y)

In [5]:
def absorbtion_cross_section(geometry,neutron_energy):
	neutron_energy=neutron_energy
	if geometry=='p':
		return pa(neutron_energy)
	else:
		return 0
	


def scattering_cross_section(geometry,neutron_energy):
	neutron_energy=neutron_energy
	if geometry=='p':
		return ps(neutron_energy)
	else:
		return 0

def fission_cross_section(geometry,neutron_energy):
	neutron_energy=neutron_energy
	if geometry=='p':
		return pf(neutron_energy)
	else:
		return 0
		
def energy_updater(energy,molar_mass):
	A=molar_mass
	E=energy
	alpha=1-((A-1)/(A+1))**2
	delta_E=np.random.uniform(0,alpha*E)
	
	return E-delta_E

In [6]:
skip=50
number_of_neutrons=10000
no_gen_sim=500				      
position_matrix=np.zeros((number_of_neutrons,3))
k_eff=np.zeros(no_gen_sim)
energy=[]
new_gen_neutron=np.zeros(no_gen_sim)
weight0=np.ones(number_of_neutrons)
weight1=np.ones(number_of_neutrons)

for i in range (number_of_neutrons):
	position_matrix[i][0]=np.random.uniform(0,6.3849)                    #radius record 
	position_matrix[i][1]=np.random.uniform(-np.pi,np.pi)                #theta record 
	position_matrix[i][2]=np.random.uniform(-np.pi/2,np.pi/2)	  		 #phi record 
	energy.append(np.random.uniform(2,4))

r=np.ones(number_of_neutrons)*position_matrix[i][0]
theta=np.ones(number_of_neutrons)*position_matrix[i][1]
phi=np.ones(number_of_neutrons)*position_matrix[i][2]
energy_new=np.ones(number_of_neutrons)*energy

k_eff=np.zeros(no_gen_sim)

In [7]:
for j in range(no_gen_sim):
		
	for i in range (number_of_neutrons):
        
        #location copy
		x=position_matrix[i][0]*math.sin(position_matrix[i][2])*math.cos(position_matrix[i][1])
		y=position_matrix[i][0]*math.sin(position_matrix[i][2])*math.sin(position_matrix[i][1])
		z=position_matrix[i][0]*math.cos(position_matrix[i][2])
		Energy = energy[i]
		alive=True
		
		while alive:
		
		# core 
			if math.sqrt(x**2+y**2+z**2)<6.3849:
				#that means the neutron is inside the core 
				
				sigma_s=scattering_cross_section("p",Energy)
				sigma_a=absorbtion_cross_section("p",Energy)
				sigma_f=fission_cross_section("p",Energy)
				
				total_sigma=sigma_s+sigma_f+sigma_a
				
				reaction_type=np.random.choice(["fission","scattering","absorbtion"],p=[sigma_f/total_sigma,sigma_s/total_sigma,sigma_a/total_sigma])
				
				#need to figure out how to make a bias here ------done
				
				if reaction_type =="scattering":
					
					random_parameter=np.random.rand()
					theta_move=np.random.uniform(0,2*np.pi)
					phi_move=np.random.uniform(0,np.pi)
					
					l= -math.log(1-random_parameter)/total_sigma
					
					x = x+l*math.sin(phi_move)*math.cos(theta_move)
					y = y+l*math.sin(phi_move)*math.sin(theta_move)
					z = z+l*math.cos(phi_move)
					
					Energy =energy_updater(Energy,235)

					if Energy<0.01:
						Energy=20
						weight1[i]=weight0[i]*(sigma_s/total_sigma)*weight1[i]
			    		
				elif reaction_type=="fission":
					
					reproduction=[1,2,3,4]
					new_neutron=np.random.choice(reproduction,p=[0.1,0.4,0.4,0.1])
					new_gen_neutron[j]=new_gen_neutron[j]+new_neutron

					weight1[i]=new_neutron*weight0[i]
					
					#location has to be saved for new initiation 
					#there will be a loop to create the same location in a matrix
					#for the same number of neutron 
					
					for k in range(1):
					
				
						r[i]=(math.sqrt(x**2+y**2+z**2))
						theta[i]=(math.atan(y/x))
						phi[i]=(math.atan(math.sqrt(x**2+y**2)/z))
						
						energy_new[i]=(np.random.uniform(9,14))
					
					alive=False 
				else:
					#russian rulete
					if weight0[i]<0.02:
						meu=np.random.rand()
						d=np.random.randint(2,10)
						if meu<=1/d:
							weight0[i]=d*weight0[i]
						else:
							alive=False
					else:
						weight1[i]=weight0[i]*(1-sigma_a/total_sigma)*weight1[i]
						alive=False 
			else:
				alive=False

	position_matrix=np.zeros((len(r),3))
	for k in range(len(r)):

		position_matrix[k][0]=r[k]
		position_matrix[k][1]=theta[k]
		position_matrix[k][2]=phi[k]

		
	energy=np.zeros(len(energy_new))
	energy=energy_new
	k_eff[j]=np.sum(weight1)/np.sum(weight0)

	print(j+1,k_eff[j])


0 1.9603964960819376
1 2.328097623519769
2 2.454798045439456
3 2.479799067558193
4 2.4924993216840248
5 2.4966995152420033
6 2.4909999020712434
7 2.4942999355237276
8 2.5175998248990297
9 2.497599886743339
10 2.5055998217177136
11 2.509099806245933
12 2.4915999180839163
13 2.5036997944910704
14 2.501099798585344
15 2.4898998000534855
16 2.504399807864734
17 2.499599777649864
18 2.494099969474587
19 2.4907999694745873
20 2.4933999380925838
21 2.4923997544019563
22 2.495799771774451
23 2.494899918869742
24 2.5026998582779743
25 2.4926998917824976
26 2.4926999207741063
27 2.492699933915765
28 2.5014999078191957
29 2.4972999049303324
30 2.497699869138859
31 2.4910999662805104
32 2.498199818369556
33 2.5051997408676936
34 2.5071998869133023
35 2.5068998383241894
36 2.4974997926317037
37 2.4906996643984276
38 2.5026997947307943
39 2.5009999211733716
40 2.49879978996131
41 2.515199854439106
42 2.5091998066421386
43 2.502699906368037
44 2.495199867458034
45 2.4892997398377252
46 2.509699885475

In [8]:
print(np.mean(k_eff[skip:]),"-\+",np.std(k_eff[skip:]))

2.499580298093311 -\+ 0.008291688033945178
